<a href="https://colab.research.google.com/github/ibrta/examen_resolution/blob/main/translated_Flight_Delay_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problème : Prédire les retards d'avion

Les objectifs de ce bloc-notes sont :
- Traiter et créer un ensemble de données à partir de fichiers .zip téléchargés
- Effectuer une analyse exploratoire des données (EDA)
- Établir un modèle de base
- Passer d'un modèle simple à un modèle d'ensemble
- Effectuer une optimisation des hyperparamètres
- Vérifier l'importance des fonctionnalités

## Introduction au scénario commercial

Vous travaillez pour un site Web de réservation de voyages qui souhaite améliorer l'expérience client pour les vols retardés. L'entreprise souhaite créer une fonctionnalité permettant aux clients de savoir si le vol sera retardé en raison des conditions météorologiques lorsqu'ils réservent un vol à destination ou en provenance des aéroports les plus fréquentés pour les voyages intérieurs aux États-Unis.

Vous êtes chargé de résoudre une partie de ce problème en utilisant l'apprentissage automatique (ML) pour identifier si le vol sera retardé en raison des conditions météorologiques. Vous avez eu accès à un ensemble de données sur les performances ponctuelles des vols intérieurs opérés par de grandes compagnies aériennes. Vous pouvez utiliser ces données pour former un modèle ML afin de prédire si le vol va être retardé pour les aéroports les plus fréquentés.

## À propos de cet ensemble de données

Cet ensemble de données contient les heures de départ et d'arrivée prévues et réelles signalées par les transporteurs aériens américains certifiés qui représentent au moins 1 % des revenus du trafic intérieur régulier de passagers. Les données ont été collectées par l'Office of Airline Information, Bureau of Transportation Statistics (BTS) des États-Unis. L'ensemble de données contient la date, l'heure, l'origine, la destination, la compagnie aérienne, la distance et l'état de retard des vols entre 2013 et 2018.

### Fonctionnalités

Pour plus d'informations sur les fonctionnalités de l'ensemble de données, consultez [Fonctionnalités de l'ensemble de données sur les retards à l'heure](https://www.transtats.bts.gov/Fields.asp).

### Attributions des ensembles de données
Site Web : https://www.transtats.bts.gov/

Les ensembles de données utilisés dans ce laboratoire ont été compilés par l'Office of Airline Information des États-Unis, Bureau of Transportation Statistics (BTS), Airline On-Time Performance Data, disponibles à l'adresse https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&DB_URL=Mode_ID=1&Mode_Desc=Aviation&Subject_ID2=0.

# Étape 1 : formulation du problème et collecte de données

Commencez ce projet en écrivant quelques phrases qui résument le problème commercial et l'objectif commercial que vous souhaitez atteindre dans ce scénario. Vous pouvez écrire vos idées dans les sections suivantes. Incluez une mesure commerciale à laquelle vous souhaitez que votre équipe aspire. Après avoir défini ces informations, rédigez l'énoncé du problème de ML. Enfin, ajoutez un ou deux commentaires sur le type de ML que cette activité représente.

#### <span style="color: blue;">Présentation du projet : incluez un résumé de ces détails dans votre présentation du projet.</span>

### 1. Déterminez si et pourquoi le ML est une solution appropriée à déployer pour ce scénario.

In [ ]:
# Écrivez votre réponse ici

### 2. Formulez le problème commercial, les indicateurs de réussite et le résultat ML souhaité.

In [ ]:
# Écrivez votre réponse ici

### 3. Identifiez le type de problème ML sur lequel vous travaillez.

In [ ]:
# Écrivez votre réponse ici

### 4. Analysez la pertinence des données avec lesquelles vous travaillez.

In [ ]:
# Écrivez votre réponse ici

### Configuration

Maintenant que vous avez décidé où vous souhaitez concentrer votre attention, vous allez configurer ce laboratoire afin de pouvoir commencer à résoudre le problème.

**Remarque :** ce bloc-notes a été créé et testé sur une instance de bloc-notes « ml.m4.xlarge » avec 25 Go de stockage.

In [ ]:
import os
from pathlib2 import Path
from zipfile import ZipFile
import time

import pandas as pd
import numpy as np
import subprocess

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
instance_type='ml.m4.xlarge'

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Étape 2 : Prétraitement et visualisation des données

Dans cette phase de prétraitement des données, vous explorez et visualisez vos données pour mieux les comprendre. Tout d'abord, importez les bibliothèques nécessaires et lisez les données dans un DataFrame pandas. Après avoir importé les données, explorez l'ensemble de données. Recherchez la forme de l'ensemble de données et explorez vos colonnes et les types de colonnes avec lesquels vous travaillerez (numériques, catégorielles). Pensez à effectuer des statistiques de base sur les entités pour avoir une idée des moyennes et des plages des entités. Examinez attentivement votre colonne cible et déterminez sa distribution.

### Questions spécifiques à prendre en compte

Tout au long de cette section du laboratoire, réfléchissez aux questions suivantes :

1. Que pouvez-vous déduire des statistiques de base que vous avez exécutées sur les entités ?

2. Que pouvez-vous déduire des distributions des classes cibles ?

3. Pouvez-vous déduire autre chose en explorant les données ?

#### <span style="color: blue;">Présentation du projet : incluez un résumé de vos réponses à ces questions (et à d’autres questions similaires) dans votre présentation de projet.</span>

Commencez par importer l’ensemble de données d’un bucket Amazon Simple Storage Service (Amazon S3) public dans cet environnement de bloc-notes.

In [ ]:
# télécharger les fichiers

zip_path = '/home/ec2-user/SageMaker/project/data/FlightDelays/'
base_path = '/home/ec2-user/SageMaker/project/data/FlightDelays/'
csv_base_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays/'

!mkdir -p {zip_path}
!mkdir -p {csv_base_path}
!aws s3 cp s3://aws-tc-largeobjects/CUR-TF-200-ACMLFO-1/flight_delay_project/data/ {zip_path} --recursive


In [ ]:
zip_files = [str(file) for file in list(Path(base_path).iterdir()) if '.zip' in str(file)]
len(zip_files)

Extraire les fichiers de valeurs séparées par des virgules (CSV) des fichiers .zip.

In [ ]:
def zip2csv(zipFile_name , file_path):
    """
    Extract csv from zip files
    zipFile_name: name of the zip file
    file_path : name of the folder to store csv
    """

    try:
        with ZipFile(zipFile_name, 'r') as z:
            print(f'Extracting {zipFile_name} ')
            z.extractall(path=file_path)
    except:
        print(f'zip2csv failed for {zipFile_name}')

for file in zip_files:
    zip2csv(file, csv_base_path)

print("Files Extracted")

In [ ]:
csv_files = [str(file) for file in list(Path(csv_base_path).iterdir()) if '.csv' in str(file)]
len(csv_files)

Avant de charger le fichier CSV, lisez le fichier HTML du dossier extrait. Ce fichier HTML inclut le contexte et des informations supplémentaires sur les entités incluses dans l'ensemble de données.

In [ ]:
from IPython.display import IFrame

IFrame(src=os.path.relpath(f"{csv_base_path}readme.html"), width=1000, height=600)

#### Charger un exemple de fichier CSV

Avant de combiner tous les fichiers CSV, examinez les données d'un seul fichier CSV. En utilisant pandas, lisez d'abord le fichier `On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_9.csv`. Vous pouvez utiliser la fonction `read_csv` intégrée dans Python ([documentation pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)).

In [ ]:
df_temp = pd.read_csv(f"{csv_base_path}On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_9.csv")

**Question** : Imprimez la longueur des lignes et des colonnes dans l'ensemble de données, ainsi que les noms des colonnes.

**Astuce** : Pour afficher les lignes et les colonnes d'un DataFrame, utilisez la fonction `<DataFrame>.shape`. Pour afficher les noms des colonnes, utilisez la fonction `<DataFrame>.columns`.

In [ ]:
df_shape = # **ENTER YOUR CODE HERE**
print(f'Rows and columns in one CSV file is {df_shape}')

**Question** : Imprimez les 10 premières lignes de l'ensemble de données.

**Astuce** : Pour imprimer un nombre « x » de lignes, utilisez la fonction intégrée « head(x) » dans pandas.

In [ ]:
# Entrez votre code ici

**Question** : Imprimez toutes les colonnes de l'ensemble de données. Pour afficher les noms des colonnes, utilisez `<DataFrame>.columns`.

In [ ]:
print(f'The column names are :')
print('#########')
for col in <CODE>:# **ENTER YOUR CODE HERE**
    print(col)

**Question** : Imprimez toutes les colonnes de l'ensemble de données qui contiennent le mot *Del*. Cela vous aidera à voir combien de colonnes contiennent des *données de retard*.

**Astuce** : Pour inclure des valeurs qui répondent à certains critères d'instruction `if`, vous pouvez utiliser une compréhension de liste Python.

Par exemple : `[x for x in [1,2,3,4,5] if x > 2]`

**Astuce** : Pour vérifier si la valeur est dans une liste, vous pouvez utiliser le mot-clé `in` ([Documentation Python in Keyword](https://www.w3schools.com/python/ref_keyword_in.asp)).

Par exemple : `5 in [1,2,3,4,5]`

In [ ]:
# Entrez votre code ici

Voici quelques questions supplémentaires pour vous aider à en savoir plus sur votre ensemble de données.

**Questions**

1. Combien de lignes et de colonnes l'ensemble de données comporte-t-il ?
2. Combien d'années sont incluses dans l'ensemble de données ?
3. Quelle est la plage de dates de l'ensemble de données ?
4. Quelles compagnies aériennes sont incluses dans l'ensemble de données ?
5. Quels aéroports d'origine et de destination sont couverts ?

**Conseils**
- Pour afficher les dimensions du DataFrame, utilisez `df_temp.shape`.
- Pour faire référence à une colonne spécifique, utilisez `df_temp.columnName` (par exemple, `df_temp.CarrierDelay`).
- Pour obtenir des valeurs uniques pour une colonne, utilisez `df_temp.column.unique()` (par exemple, `df_temp.Year.unique()`).

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", <CODE>)
print("The months covered in this dataset are: ", <CODE>)
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

**Question** : Quel est le nombre total d'aéroports d'origine et de destination ?

**Astuce** : Pour trouver les valeurs de chaque aéroport en utilisant les colonnes **Origin** et **Dest**, vous pouvez utiliser la fonction `values_count` dans pandas ([documentation pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)).

In [ ]:
counts = pd.DataFrame({'Origin':<CODE>, 'Destination':<CODE>})
counts

**Question** : Imprimez les 15 principaux aéroports d'origine et de destination en fonction du nombre de vols dans l'ensemble de données.

**Astuce** : Vous pouvez utiliser la fonction `sort_values` dans pandas ([documentation pandas.DataFrame.sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)).

In [ ]:
counts.sort_values(by=<CODE>,ascending=False).head(15) # Enter your code here

**Compte tenu de toutes les informations sur un voyage en avion, pouvez-vous prédire s'il sera retardé ?**

La colonne **ArrDel15** est une variable indicatrice qui prend la valeur *1* lorsque le retard est supérieur à 15 minutes. Sinon, elle prend la valeur *0*.

Vous pouvez l'utiliser comme colonne cible pour le problème de classification.

Supposons maintenant que vous voyagez de San Francisco à Los Angeles pour un voyage d'affaires. Vous souhaitez mieux gérer vos réservations à Los Angeles. Vous souhaitez donc avoir une idée du retard éventuel de votre vol, compte tenu d'un ensemble de caractéristiques. Combien de caractéristiques de cet ensemble de données devez-vous connaître avant votre vol ?

Des colonnes telles que « DepDelay », « ArrDelay », « CarrierDelay », « WeatherDelay », « NASDelay », « SecurityDelay », « LateAircraftDelay » et « DivArrDelay » contiennent des informations sur un retard. Mais ce retard peut avoir eu lieu à l'origine ou à la destination. S'il y avait un retard soudain dû aux conditions météorologiques 10 minutes avant l'atterrissage, ces données ne seraient pas utiles pour gérer vos réservations à Los Angeles.

Ainsi, pour simplifier l'énoncé du problème, considérez les colonnes suivantes pour prédire un retard d'arrivée : <br>

`Year`, `Quarter`, `Month`, `DayofMonth`, `DayOfWeek`, `FlightDate`, `Reporting_Airline`, `Origin`, `OriginState`, `Dest`, `DestState`, `CRSDepTime`, `DepDelayMinutes`, `DepartureDelayGroups`, `Cancelled`, `Diverted`, `Distance`, `DistanceGroup`, `ArrDelay`, `ArrDelayMinutes`, `ArrDel15`, `AirTime`

Vous filtrerez également les aéroports source et de destination comme suit :
- Principaux aéroports : ATL, ORD, DFW, DEN, CLT, LAX, IAH, PHX, SFO
- Principaux cinq compagnies aériennes : UA, OO, WN, AA, DL

Ces informations devraient permettre de réduire la taille des données dans les fichiers CSV qui seront combinés.

#### Combiner tous les fichiers CSV

Tout d'abord, créez un DataFrame vide que vous utiliserez pour copier vos DataFrames individuels à partir de chaque fichier. Ensuite, pour chaque fichier de la liste `csv_files` :

1. Lisez le fichier CSV dans un dataframe
2. Filtrez les colonnes en fonction de la variable `filter_cols`

```
columns = ['col1', 'col2']
df_filter = df[columns]
```

3. Ne conservez que les `subset_vals` dans chacun des `subset_cols`. Pour vérifier si le `val` se trouve dans la colonne DataFrame, utilisez la fonction `isin` dans pandas ([documentation pandas.DataFram.isin](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)). Ensuite, choisissez les lignes qui l'incluent.

```
df_eg[df_eg['col1'].isin('5')]
```

4. Concaténez le DataFrame avec le DataFrame vide

In [ ]:
def combine_csv(csv_files, filter_cols, subset_cols, subset_vals, file_name):

    """
    Combine csv files into one Data Frame
    csv_files: list of csv file paths
    filter_cols: list of columns to filter
    subset_cols: list of columns to subset rows
    subset_vals: list of list of values to subset rows
    """

    df = pd.DataFrame()

    for file in csv_files:
        df_temp = pd.read_csv(file)
        df_temp = df_temp[filter_cols]
        for col, val in zip(subset_cols,subset_vals):
            df_temp = df_temp[df_temp[col].isin(val)]

        df = pd.concat([df, df_temp], axis=0)

    df.to_csv(file_name, index=False)
    print(f'Combined csv stored at {file_name}')

In [ ]:
# cols est la liste des colonnes pour prédire le retard d'arrivée
cols = ['Year','Quarter','Month','DayofMonth','DayOfWeek','FlightDate',
        'Reporting_Airline','Origin','OriginState','Dest','DestState',
        'CRSDepTime','Cancelled','Diverted','Distance','DistanceGroup',
        'ArrDelay','ArrDelayMinutes','ArrDel15','AirTime']

subset_cols = ['Origin', 'Dest', 'Reporting_Airline']

# subset_vals est une collection de listes des principaux aéroports d'origine et de destination et des 5 principales compagnies aériennes
subset_vals = [['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'],
               ['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'],
               ['UA', 'OO', 'WN', 'AA', 'DL']]

Utilisez la fonction précédente pour fusionner tous les différents fichiers en un seul fichier que vous pouvez lire facilement.

**Remarque** : ce processus prendra 5 à 7 minutes.

In [ ]:
start = time.time()
combined_csv_filename = f"{base_path}combined_files.csv"
combine_csv(csv_files, cols, subset_cols, subset_vals, combined_csv_filename)
print(f'CSVs merged in {round((time.time() - start)/60,2)} minutes')

#### Charger l'ensemble de données

Charger l'ensemble de données combiné.

In [ ]:
data = pd.read_csv(combined_csv_filename)

Imprimez les cinq premiers enregistrements.

In [ ]:
# Entrez votre code ici

Voici quelques questions supplémentaires pour vous aider à en savoir plus sur votre ensemble de données.

**Questions**

1. Combien de lignes et de colonnes l'ensemble de données comporte-t-il ?
2. Combien d'années sont incluses dans l'ensemble de données ?
3. Quelle est la plage de dates de l'ensemble de données ?
4. Quelles compagnies aériennes sont incluses dans l'ensemble de données ?
5. Quels aéroports d'origine et de destination sont couverts ?

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", list(<CODE>))
print("The months covered in this dataset are: ", sorted(list(<CODE>)))
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

Définissez votre colonne cible : **is_delay** (*1* signifie que l'heure d'arrivée a été retardée de plus de 15 minutes et *0* signifie tous les autres cas). Pour renommer la colonne de **ArrDel15** à *is_delay*, utilisez la méthode `rename`.

**Astuce** : vous pouvez utiliser la fonction `rename` dans pandas ([documentation pandas.DataFrame.rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)).

Par exemple :
```
data.rename(columns={'col1':'column1'}, inplace=True)
```

In [ ]:
data.rename(columns=<CODE>, inplace=True) # Enter your code here

Recherchez les valeurs nulles dans les colonnes. Vous pouvez utiliser la fonction `isnull()` ([documentation pandas.isnull](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.isnull.html)).

**Astuce** : `isnull()` détecte si la valeur particulière est nulle ou non. Elle renvoie un booléen (*True* ou *False*) à sa place. Pour additionner le nombre de colonnes, utilisez la fonction `sum(axis=0)` (par exemple, `df.isnull().sum(axis=0)`).

In [ ]:
# Entrez votre code ici

Les détails du retard d'arrivée et du temps d'antenne sont manquants pour 22 540 lignes sur 1 658 130, ce qui représente 1,3 %. Vous pouvez supprimer ou imputer ces lignes. La documentation ne mentionne aucune information sur les lignes manquantes.

In [ ]:
# ## Supprimer les colonnes nulles
data = data[~data.is_delay.isnull()]
data.isnull().sum(axis = 0)

Obtenez l'heure du jour au format 24 heures à partir de CRSDepTime.

In [ ]:
data['DepHourofDay'] = (data['CRSDepTime']//100)

## **L'énoncé du problème ML**
- Étant donné un ensemble de fonctionnalités, pouvez-vous prédire si un vol va être retardé de plus de 15 minutes ?
- Étant donné que la variable cible ne prend qu'une valeur de *0* ou *1*, vous pouvez utiliser un algorithme de classification.

Avant de commencer la modélisation, il est recommandé d'examiner la distribution des fonctionnalités, les corrélations et autres.
- Cela vous donnera une idée de toute non-linéarité ou de tout modèle dans les données
- Modèles linéaires : ajoutez des fonctionnalités de puissance, exponentielles ou d'interaction
- Essayez un modèle non linéaire
- Déséquilibre des données
- Choisissez des mesures qui ne donneront pas de performances de modèle biaisées (précision par rapport à l'aire sous la courbe, ou AUC)
- Utilisez des fonctions de perte pondérées ou personnalisées
- Données manquantes
- Effectuez une imputation basée sur des statistiques simples : moyenne, médiane, mode (variables numériques), classe fréquente (variables catégorielles)
- Imputation basée sur le clustering (k-voisins les plus proches, ou KNN, pour prédire la valeur de la colonne)
- Supprimer la colonne

### Exploration des données

Vérifiez les classes *délai* par rapport à *pas de délai*.

In [ ]:
(data.groupby('is_delay').size()/len(data) ).plot(kind='bar')# Enter your code here
plt.ylabel('Frequency')
plt.title('Distribution of classes')
plt.show()

**Question** : Que pouvez-vous déduire du graphique à barres concernant le rapport entre *délai* et *absence de délai* ?

In [ ]:
# Entrez votre réponse ici

Exécutez les deux cellules suivantes et répondez aux questions.

In [ ]:
viz_columns = ['Month', 'DepHourofDay', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest']
fig, axes = plt.subplots(3, 2, figsize=(20,20), squeeze=False)
# fig.autofmt_xdate(rotation=90)

for idx, column in enumerate(viz_columns):
    ax = axes[idx//2, idx%2]
    temp = data.groupby(column)['is_delay'].value_counts(normalize=True).rename('percentage').\
    mul(100).reset_index().sort_values(column)
    sns.barplot(x=column, y="percentage", hue="is_delay", data=temp, ax=ax)
    plt.ylabel('% delay/no-delay')


plt.show()

In [ ]:
sns.lmplot( x="is_delay", y="Distance", data=data, fit_reg=False, hue='is_delay', legend=False)
plt.legend(loc='center')
plt.xlabel('is_delay')
plt.ylabel('Distance')
plt.show()

**Questions**

À l'aide des données des graphiques précédents, répondez à ces questions :

- Quels mois connaissent le plus de retards ?
- À quelle heure de la journée connaît le plus de retards ?
- Quel jour de la semaine connaît le plus de retards ?
- Quelle compagnie aérienne connaît le plus de retards ?
- Quels aéroports d'origine et de destination connaissent le plus de retards ?
- La distance du vol est-elle un facteur dans les retards ?

In [ ]:
# Entrez vos réponses ici

### Fonctionnalités

Regardez toutes les colonnes et quels sont leurs types spécifiques.

In [ ]:
data.columns

In [ ]:
data.dtypes

Filtrage des colonnes requises :
- *Date* est redondant, car vous avez *Year*, *Quarter*, *Month*, *DayofMonth* et *DayOfWeek* pour décrire la date.
- Utilisez les codes *Origin* et *Dest* au lieu de *OriginState* et *DestState*.
- Étant donné que vous classez uniquement si le vol est retardé ou non, vous n'avez pas besoin de *TotalDelayMinutes*, *DepDelayMinutes* et *ArrDelayMinutes*.

Traitez *DepHourofDay* comme une variable catégorielle, car elle n'a aucune relation quantitative avec la cible.
- Si vous deviez effectuer un codage one-hot de cette variable, cela donnerait 23 colonnes supplémentaires.
- D'autres alternatives à la gestion des variables catégorielles incluent le codage par hachage, le codage par moyenne régularisée et la segmentation des valeurs, entre autres.
- Dans ce cas, vous n'avez besoin que de diviser en compartiments.

Pour changer le type de colonne en catégorie, utilisez la fonction `astype` ([documentation pandas.DataFrame.astype](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html)).

In [ ]:
data_orig = data.copy()
data = data[[ 'is_delay', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay']]
categorical_columns  = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest', 'DepHourofDay']
for c in categorical_columns:
    data[c] = data[c].astype('category')

Pour utiliser l'encodage one-hot, utilisez la fonction `get_dummies` dans pandas pour les colonnes catégorielles que vous avez sélectionnées. Ensuite, vous pouvez concaténer ces entités générées à votre jeu de données d'origine en utilisant la fonction `concat` dans pandas. Pour encoder des variables catégorielles, vous pouvez également utiliser un *encodage factice* en utilisant un mot-clé `drop_first=True`. Pour plus d'informations sur l'encodage factice, voir [Variable factice (statistiques)](https://en.wikiversity.org/wiki/Dummy_variable_(statistics)).

Par exemple :
```
pd.get_dummies(df[['column1','columns2']], drop_first=True)
```

In [ ]:
data_dummies = pd.get_dummies(<CODE>, drop_first=True) # Enter your code here
data_dummies = data_dummies.replace({True: 1, False: 0})
data = pd.concat([<CODE>, <CODE>], axis = 1)
data.drop(categorical_columns,axis=1, inplace=True)

Vérifiez la longueur de l'ensemble de données et les nouvelles colonnes.

**Astuce** : utilisez les propriétés `shape` et `columns`.

In [ ]:
# Entrez votre code ici

In [ ]:
# Entrez votre code ici

Vous êtes maintenant prêt à entraîner le modèle. Avant de fractionner les données, renommez la colonne **is_delay** en *target*.

**Astuce** : vous pouvez utiliser la fonction `rename` dans pandas ([documentation pandas.DataFrame.rename](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)).

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# Enter your code here

## <span style="color:red"> Fin de l'étape 2 </span>

Enregistrez le fichier de projet sur votre ordinateur local. Suivez ces étapes :

1. Dans l'explorateur de fichiers à gauche, cliquez avec le bouton droit sur le bloc-notes sur lequel vous travaillez.

2. Choisissez **Télécharger** et enregistrez le fichier localement.

Cette action télécharge le bloc-notes actuel dans le dossier de téléchargement par défaut de votre ordinateur.

# Étape 3 : formation et évaluation du modèle

Vous devez inclure certaines étapes préliminaires lorsque vous convertissez l'ensemble de données d'un DataFrame vers un format qu'un algorithme d'apprentissage automatique peut utiliser. Pour Amazon SageMaker, vous devez effectuer ces étapes :

1. Divisez les données en `train_data`, `validation_data` et `test_data` à l'aide de `sklearn.model_selection.train_test_split`.

2. Convertissez l'ensemble de données dans un format de fichier approprié que la tâche de formation Amazon SageMaker peut utiliser. Il peut s'agir d'un fichier CSV ou d'un enregistrement protobuf. Pour plus d'informations, consultez [Formats de données courants pour la formation](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html).

3. Téléchargez les données dans votre compartiment S3. Si vous n'en avez pas encore créé, consultez [Créer un bucket](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html).

Utilisez les cellules suivantes pour effectuer ces étapes. Insérez et supprimez des cellules si nécessaire.

#### <span style="color: blue;">Présentation du projet : dans votre présentation de projet, notez les décisions clés que vous avez prises au cours de cette phase.</span>

### Séparation des trains et des tests

In [ ]:
from sklearn.model_selection import train_test_split
def split_data(data):
    train, test_and_validate = train_test_split(data, test_size=0.2, random_state=42, stratify=data['target'])
    test, validate = train_test_split(test_and_validate, test_size=0.5, random_state=42, stratify=test_and_validate['target'])
    return train, validate, test

In [ ]:
train, validate, test = split_data(data)
print(train['target'].value_counts())
print(test['target'].value_counts())
print(validate['target'].value_counts())

**Exemple de réponse**
```
0,0 1033570
1,0 274902
Nom : cible, type de données : int64
0,0 129076
1,0 34483
Nom : cible, type de données : int64
0,0 129612
1,0 33947
Nom : cible, type de données : int64
```

### Modèle de classification de base

In [ ]:
import sagemaker
from sagemaker.serializers import CSVSerializer
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

# Instanciez l'objet estimateur LinearLearner avec 1 ml.m4.xlarge
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               instance_count=<CODE>,
                                               instance_type=<CODE>,
                                               predictor_type=<CODE>,
                                               binary_classifier_model_selection_criteria=<CODE>)

### Exemple de code
```
num_classes = len(pd.unique(train_labels))
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
instance_count=1,
instance_type='ml.m4.xlarge',
predictor_type='binary_classifier',
binary_classifier_model_selection_criteria = 'cross_entropy_loss')

```

L'apprenant linéaire accepte les données de formation dans les types de contenu protobuf ou CSV. Il accepte également les demandes d'inférence dans les types de contenu protobuf, CSV ou JavaScript Object Notation (JSON). Les données de formation ont des fonctionnalités et des étiquettes de vérité fondamentale, mais les données d'une demande d'inférence n'ont que des fonctionnalités.

Dans un pipeline de production, AWS recommande de convertir les données au format protobuf d'Amazon SageMaker et de les stocker dans Amazon S3. Pour être opérationnel rapidement, AWS fournit l'opération `record_set` pour convertir et télécharger l'ensemble de données lorsqu'il est suffisamment petit pour tenir dans la mémoire locale. Il accepte les tableaux NumPy comme ceux que vous avez déjà, vous l'utiliserez donc pour cette étape. L'objet `RecordSet` suivra l'emplacement Amazon S3 temporaire de vos données. Créez des enregistrements de formation, de validation et de test à l'aide de la fonction `estimator.record_set`. Ensuite, démarrez votre tâche de formation à l'aide de la fonction `estimator.fit`.

In [ ]:
# ## Créer des enregistrements de train, de validation et de test
train_records = classifier_estimator.record_set(train.values[:, 1:].astype(np.float32), train.values[:, 0].astype(np.float32), channel='train')
val_records = classifier_estimator.record_set(validate.values[:, 1:].astype(np.float32), validate.values[:, 0].astype(np.float32), channel='validation')
test_records = classifier_estimator.record_set(test.values[:, 1:].astype(np.float32), test.values[:, 0].astype(np.float32), channel='test')

Entraînez maintenant votre modèle sur l'ensemble de données que vous venez de télécharger.

### Exemple de code
```
linear.fit([train_records,val_records,test_records])
```

In [ ]:
# ## Ajuster le classificateur
# Entrez votre code ici

## Évaluation du modèle
Dans cette section, vous allez évaluer votre modèle formé.

Tout d’abord, examinez les métriques de la tâche de formation :

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(classifier_estimator._current_job_name,
                                         metric_names = ['test:objective_loss',
                                                         'test:binary_f_beta',
                                                         'test:precision',
                                                         'test:recall']
                                        ).dataframe()

Ensuite, configurez certaines fonctions qui vous aideront à charger les données de test dans Amazon S3 et à effectuer une prédiction à l'aide de la fonction de prédiction par lots. L'utilisation de la prédiction par lots contribuera à réduire les coûts, car les instances ne s'exécuteront que lorsque des prédictions seront effectuées sur les données de test fournies.

**Remarque :** remplacez `<LabBucketName>` par le nom du compartiment de laboratoire créé lors de la configuration du laboratoire.

In [ ]:
import io
# bucket='<NomLabBucket>'
prefix='flight-linear'
train_file='flight_train.csv'
test_file='flight_test.csv'
validate_file='flight_validate.csv'
whole_file='flight.csv'
s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

In [ ]:
def batch_linear_predict(test_data, estimator):
    batch_X = test_data.iloc[:,1:];
    batch_X_file='batch-in.csv'
    upload_s3_csv(batch_X_file, 'batch-in', batch_X)

    batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
    batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

    classifier_transformer = estimator.transformer(instance_count=1,
                                           instance_type='ml.m4.xlarge',
                                           strategy='MultiRecord',
                                           assemble_with='Line',
                                           output_path=batch_output)

    classifier_transformer.transform(data=batch_input,
                             data_type='S3Prefix',
                             content_type='text/csv',
                             split_type='Line')

    classifier_transformer.wait()

    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
    target_predicted_df = pd.read_json(io.BytesIO(obj['Body'].read()),orient="records",lines=True)
    return test_data.iloc[:,0], target_predicted_df.iloc[:,0]

Pour exécuter les prédictions sur l'ensemble de données de test, exécutez la fonction `batch_linear_predict` (qui a été définie précédemment) sur votre ensemble de données de test.

In [ ]:
test_labels, target_predicted = batch_linear_predict(test, classifier_estimator)

Pour afficher un graphique de la matrice de confusion et diverses mesures de notation, créez quelques fonctions :

In [ ]:
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(test_labels, target_predicted):
    matrix = confusion_matrix(test_labels, target_predicted)
    df_confusion = pd.DataFrame(matrix)
    colormap = sns.color_palette("BrBG", 10)
    sns.heatmap(df_confusion, annot=True, fmt='.2f', cbar=None, cmap=colormap)
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.ylabel("True Class")
    plt.xlabel("Predicted Class")
    plt.show()


In [ ]:
from sklearn import metrics

def plot_roc(test_labels, target_predicted):
    TN, FP, FN, TP = confusion_matrix(test_labels, target_predicted).ravel()
# Sensibilité, taux de réussite, rappel ou taux de vrais positifs
    Sensitivity  = float(TP)/(TP+FN)*100
# Spécificité ou taux de vrais négatifs
    Specificity  = float(TN)/(TN+FP)*100
# Précision ou valeur prédictive positive
    Precision = float(TP)/(TP+FP)*100
# Valeur prédictive négative
    NPV = float(TN)/(TN+FN)*100
# Taux de faux positifs ou de chute de résultats
    FPR = float(FP)/(FP+TN)*100
# Taux de faux négatifs
    FNR = float(FN)/(TP+FN)*100
# Taux de fausses découvertes
    FDR = float(FP)/(TP+FP)*100
# Précision globale
    ACC = float(TP+TN)/(TP+FP+FN+TN)*100

    print("Sensitivity or TPR: ", Sensitivity, "%")
    print( "Specificity or TNR: ",Specificity, "%")
    print("Precision: ",Precision, "%")
    print("Negative Predictive Value: ",NPV, "%")
    print( "False Positive Rate: ",FPR,"%")
    print("False Negative Rate: ",FNR, "%")
    print("False Discovery Rate: ",FDR, "%" )
    print("Accuracy: ",ACC, "%")

    test_labels = test.iloc[:,0];
    print("Validation AUC", metrics.roc_auc_score(test_labels, target_predicted) )

    fpr, tpr, thresholds = metrics.roc_curve(test_labels, target_predicted)
    roc_auc = metrics.auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % (roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")

# créer l'axe des seuils (scores)
    ax2 = plt.gca().twinx()
    ax2.plot(fpr, thresholds, markeredgecolor='r',linestyle='dashed', color='r')
    ax2.set_ylabel('Threshold',color='r')
    ax2.set_ylim([thresholds[-1],thresholds[0]])
    ax2.set_xlim([fpr[0],fpr[-1]])

    print(plt.figure())

Pour tracer la matrice de confusion, appelez la fonction `plot_confusion_matrix` sur les données `test_labels` et `target_predicted` de votre tâche par lots :

In [ ]:
# Entrez votre code ici

### Questions clés à prendre en compte :

1. Comment les performances de votre modèle sur l'ensemble de test se comparent-elles à ses performances sur l'ensemble d'entraînement ? Que pouvez-vous déduire de cette comparaison ?
2. Existe-t-il des différences évidentes entre les résultats de mesures telles que l'exactitude, la précision et le rappel ? Si oui, pourquoi constatez-vous ces différences ?
3. Compte tenu de la situation et des objectifs de votre entreprise, quelle mesure (ou quelles mesures) est la plus importante à prendre en compte ? Pourquoi ?
4. D'un point de vue commercial, le résultat de la mesure (ou des mesures) que vous considérez comme la plus importante est-il suffisant pour ce dont vous avez besoin ? Si ce n'est pas le cas, quels sont les éléments que vous pourriez modifier dans votre prochaine itération ? (Cela se produira dans la section sur l'ingénierie des fonctionnalités, qui est la suivante.)

Utilisez les cellules suivantes pour répondre à ces questions (et à d'autres). Insérez et supprimez des cellules si nécessaire.

#### <span style="color: blue;">Présentation du projet : dans votre présentation du projet, notez vos réponses à ces questions (et à d'autres questions similaires auxquelles vous pourriez répondre) dans cette section. Enregistrez les détails clés et les décisions que vous avez prises.

**Question** : Que pouvez-vous résumer à partir de la matrice de confusion ?

In [ ]:
# Entrez votre réponse ici

## <span style="color:red"> Fin de l'étape 3 </span>

Enregistrez le fichier de projet sur votre ordinateur local. Suivez ces étapes :

1. Dans l'explorateur de fichiers à gauche, cliquez avec le bouton droit sur le bloc-notes sur lequel vous travaillez.

2. Sélectionnez **Télécharger** et enregistrez le fichier localement.

Cette action télécharge le bloc-notes actuel dans le dossier de téléchargement par défaut de votre ordinateur.

# Itération II

# Étape 4 : Ingénierie des fonctionnalités

Vous avez maintenant effectué une itération de formation et d'évaluation de votre modèle. Étant donné que le premier résultat que vous avez obtenu pour votre modèle n'était probablement pas suffisant pour résoudre votre problème métier, que pourriez-vous modifier dans vos données pour éventuellement améliorer les performances du modèle ?

### Questions clés à prendre en compte :

1. Comment l'équilibre de vos deux classes principales (*délai* et *pas de délai*) pourrait-il avoir un impact sur les performances du modèle ?
2. Avez-vous des fonctionnalités corrélées ?
3. À ce stade, pourriez-vous effectuer des techniques de réduction des fonctionnalités qui pourraient avoir un impact positif sur les performances du modèle ?
4. Pouvez-vous envisager d'ajouter d'autres données ou ensembles de données ?
5. Après avoir effectué une ingénierie des fonctionnalités, comment les performances de votre modèle se comparent-elles à la première itération ?

Utilisez les cellules suivantes pour effectuer des techniques d'ingénierie des fonctionnalités spécifiques qui, selon vous, pourraient améliorer les performances de votre modèle (utilisez les questions précédentes comme guide). Insérez et supprimez des cellules si nécessaire.

#### <span style="color: blue;">Présentation du projet : dans votre présentation du projet, enregistrez vos décisions clés et les méthodes que vous utilisez dans cette section. Incluez également toutes les nouvelles mesures de performance que vous obtenez après avoir réévalué votre modèle.</span>

Avant de commencer, réfléchissez à la raison pour laquelle la précision et le rappel sont d'environ 80 % et l'exactitude de 99 %.

Ajoutez d'autres fonctionnalités :

1. Vacances
2. Météo

Comme la liste des jours fériés de 2014 à 2018 est connue, vous pouvez créer une variable indicatrice **is_holiday** pour les marquer.

L'hypothèse est que les retards d'avion pourraient être plus élevés pendant les jours fériés par rapport aux autres jours. Ajoutez une variable booléenne `is_holiday` qui inclut les jours fériés des années 2014-2018.

In [ ]:
# Source : http://www.calendarpedia.com/holidays/f%C3%A9ri%C3%A9s-federaux-2014.html

holidays_14 = ['2014-01-01',  '2014-01-20', '2014-02-17', '2014-05-26', '2014-07-04', '2014-09-01', '2014-10-13', '2014-11-11', '2014-11-27', '2014-12-25' ]
holidays_15 = ['2015-01-01',  '2015-01-19', '2015-02-16', '2015-05-25', '2015-06-03', '2015-07-04', '2015-09-07', '2015-10-12', '2015-11-11', '2015-11-26', '2015-12-25']
holidays_16 = ['2016-01-01',  '2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25', '2016-12-26']
holidays_17 = ['2017-01-02', '2017-01-16', '2017-02-20', '2017-05-29' , '2017-07-04', '2017-09-04' ,'2017-10-09', '2017-11-10', '2017-11-23', '2017-12-25']
holidays_18 = ['2018-01-01', '2018-01-15', '2018-02-19', '2018-05-28' , '2018-07-04', '2018-09-03' ,'2018-10-08', '2018-11-12','2018-11-22', '2018-12-25']
holidays = holidays_14+ holidays_15+ holidays_16 + holidays_17+ holidays_18

# ## Ajouter une variable indicatrice pour les jours fériés
data_orig['is_holiday'] = # Enter your code here

Les données météorologiques ont été extraites de https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&startDate=2014-01-01&endDate=2018-12-31.
<br>

Cet ensemble de données contient des informations sur la vitesse du vent, les précipitations, la neige et la température pour les villes par code d'aéroport.

**Question** : Le mauvais temps dû à la pluie, aux vents violents ou à la neige peut-il entraîner des retards d'avion ? Vous allez maintenant vérifier.

In [ ]:
!aws s3 cp s3://aws-tc-largeobjects/CUR-TF-200-ACMLFO-1/flight_delay_project/data2/daily-summaries.csv /home/ec2-user/SageMaker/project/data/
# !wget 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&startDate=2014-01-01&endDate=2018-12-31' -O /home/ec2-user/SageMaker/project/data/daily-summaries.csv

Importez les données météorologiques préparées pour les codes d'aéroport dans l'ensemble de données. Utilisez les stations et aéroports suivants pour l'analyse. Créez une nouvelle colonne appelée *aéroport* qui associe la station météo au nom de l'aéroport.

In [ ]:
weather = pd.read_csv('/home/ec2-user/SageMaker/project/data/daily-summaries.csv')
station = ['USW00023174','USW00012960','USW00003017','USW00094846','USW00013874','USW00023234','USW00003927','USW00023183','USW00013881']
airports = ['LAX', 'IAH', 'DEN', 'ORD', 'ATL', 'SFO', 'DFW', 'PHX', 'CLT']

# ## Cartographier les stations météorologiques selon le code de l'aéroport
station_map = {s:a for s,a in zip(station, airports)}
weather['airport'] = weather['STATION'].map(station_map)

À partir de la colonne **DATE**, créez une autre colonne appelée *MOIS*.

In [ ]:
weather['MONTH'] = weather['DATE'].apply(lambda x: x.split('-')[1])
weather.head()

### Exemple de sortie
```
  STATION DATE AWND PRCP NEIGE SNWD TAVG TMAX TMIN aéroport MOIS
0 USW00023174 01/01/2014 16 0 NaN NaN 131,0 178,0 78,0 LAX 01
1 USW00023174 02/01/2014 22 0 NaN NaN 159,0 256,0 100,0 LAX 01
2 USW00023174 03/01/2014 17 0 NaN NaN 140,0 178,0 83,0 LAX 01
3 USW00023174 04/01/2014 18 0 NaN NaN 136,0 183,0 100,0 LAX 01
4 USW00023174 05/01/2014 18 0 NaN NaN 151,0 244,0 83,0 LAX 01
```

Analysez et gérez les colonnes **SNOW** et **SNWD** pour les valeurs manquantes en utilisant `fillna()`. Pour vérifier les valeurs manquantes pour toutes les colonnes, utilisez la fonction `isna()`.

In [ ]:
weather.SNOW.fillna(0, inplace=True)
weather.SNWD.fillna(0, inplace=True)
weather.isna().sum()

**Question** : Imprimez l'index des lignes qui ont des valeurs manquantes pour *TAVG*, *TMAX*, *TMIN*.

**Astuce** : Pour trouver les lignes manquantes, utilisez la fonction `isna()`. Ensuite, pour obtenir l'index, utilisez la liste sur la variable *idx*.

In [ ]:
idx = np.array([i for i in range(len(weather))])
TAVG_idx = idx[weather.TAVG.isna()]
TMAX_idx = # Enter your code here
TMIN_idx = # Enter your code here
TAVG_idx

### Exemple de sortie

```
array([ 3956, 3957, 3958, 3959, 3960, 3961, 3962, 3963, 3964,
3965, 3966, 3967, 3968, 3969, 3970, 3971, 3972, 3973,
3974, 3975, 3976, 3977, 3978, 3979, 3980, 3981, 3982,
3983, 3984, 3985, 4017, 4018, 4019, 4020, 4021, 4022,
4023,  4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031,
        4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040,
        4041, 4042, 4043, 4044, 4045, 4046, 4047, 13420])
```

Vous pouvez remplacer les valeurs *TAVG*, *TMAX* et *TMIN* manquantes par la valeur moyenne d'une gare ou d'un aéroport particulier. Étant donné que des lignes consécutives de *TAVG_idx* manquent, il ne serait pas possible de les remplacer par une valeur précédente. Remplacez-les plutôt par la moyenne. Utilisez la fonction `groupby` pour agréger les variables avec une valeur moyenne.

**Astuce :** regroupez par `MONTH` et `STATION`.

In [ ]:
weather_impute = weather.groupby([<CODE>]).agg({'TAVG':'mean','TMAX':'mean', 'TMIN':'mean' }).reset_index()# Enter your code here
weather_impute.head(2)

Fusionner les données moyennes avec les données météorologiques.

In [ ]:

weather = pd.merge(weather, weather_impute,  how='left', left_on=['MONTH','STATION'], right_on = ['MONTH','STATION'])\
.rename(columns = {'TAVG_y':'TAVG_AVG',
                   'TMAX_y':'TMAX_AVG',
                   'TMIN_y':'TMIN_AVG',
                   'TAVG_x':'TAVG',
                   'TMAX_x':'TMAX',
                   'TMIN_x':'TMIN'})

Vérifiez à nouveau les valeurs manquantes.

In [ ]:
weather.TAVG[TAVG_idx] = weather.TAVG_AVG[TAVG_idx]
weather.TMAX[TMAX_idx] = weather.TMAX_AVG[TMAX_idx]
weather.TMIN[TMIN_idx] = weather.TMIN_AVG[TMIN_idx]
weather.isna().sum()

Supprimez `STATION,MONTH,TAVG_AVG,TMAX_AVG,TMIN_AVG,TMAX,TMIN,SNWD` de l'ensemble de données.

In [ ]:
weather.drop(columns=['STATION','MONTH','TAVG_AVG', 'TMAX_AVG', 'TMIN_AVG', 'TMAX' ,'TMIN', 'SNWD'],inplace=True)

Ajoutez les conditions météorologiques d’origine et de destination à l’ensemble de données.

In [ ]:
# ## Ajouter les conditions météorologiques d'origine
data_orig = pd.merge(data_orig, weather,  how='left', left_on=['FlightDate','Origin'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_O','PRCP':'PRCP_O', 'TAVG':'TAVG_O', 'SNOW': 'SNOW_O'})\
.drop(columns=['DATE','airport'])

# ## Ajouter les conditions météorologiques de destination
data_orig = pd.merge(data_orig, weather,  how='left', left_on=['FlightDate','Dest'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_D','PRCP':'PRCP_D', 'TAVG':'TAVG_D', 'SNOW': 'SNOW_D'})\
.drop(columns=['DATE','airport'])

**Remarque ** : il est toujours recommandé de vérifier les valeurs nulles ou les valeurs NA après les jointures.

In [ ]:
sum(data.isna().any())

In [ ]:
data_orig.columns

Convertissez les données catégorielles en données numériques en utilisant le codage one-hot.

In [ ]:
data = data_orig.copy()
data = data[['is_delay', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay','is_holiday', 'AWND_O', 'PRCP_O',
       'TAVG_O', 'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D']]


categorical_columns  = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest', 'is_holiday']
for c in categorical_columns:
    data[c] = data[c].astype('category')

In [ ]:
data_dummies = pd.get_dummies(data[['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest', 'is_holiday']], drop_first=True)
data_dummies = data_dummies.replace({True: 1, False: 0})
data = pd.concat([data, data_dummies], axis = 1)
data.drop(categorical_columns,axis=1, inplace=True)

Vérifiez les nouvelles colonnes.

In [ ]:
data.shape

In [ ]:
data.columns

### Exemple de sortie

```
Index(['Distance', 'DepHourofDay', 'is_delay', 'AWND_O', 'PRCP_O', 'TAVG_O',
'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D', 'Year_2015',
'Year_2016', 'Year_2017', 'Year_2018', 'Quarter_2', 'Quarter_3',
'Quarter_4', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
'Month_7', 'Month_8', 'Month_9', 'Mois_10', 'Mois_11', 'Mois_12',
'JourduMois_2', 'JourduMois_3', 'JourduMois_4', 'JourduMois_5',
'JourduMois_6', 'JourduMois_7', 'JourduMois_8', 'JourduMois_9',
'JourduMois_10', 'JourduMois_11', 'JourduMois_12', 'JourduMois_13',
'JourduMois_14', 'JourduMois_15', 'JourduMois_16', 'JourduMois_17',
'JourduMois_18', 'JourduMois_19', 'JourduMois_20', 'JourduMois_21',
       'JourduMois_22', 'JourduMois_23', 'JourduMois_24', 'JourduMois_25',
       'JourduMois_26', 'JourduMois_27', 'JourduMois_28', 'JourduMois_29',
       'JourduMois_30', 'JourduMois_31', 'Jour de la Semaine_2', 'Jour de la Semaine_3',
       « Jour de la semaine_4 », « Jour de la semaine_5 », « Jour de la semaine_6 », « Jour de la semaine_7 »,
       'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA',
       'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origine_DFW',
'Origine_IAH', 'Origine_LAX', 'Origine_ORD', 'Origine_PHX', 'Origine_SFO',
'Dest_CLT', 'Dest_DEN', 'Dest_DFW', 'Dest_IAH', 'Dest_LAX', 'Dest_ORD',
'Dest_PHX', 'Dest_SFO', 'is_holiday_1'],
dtype='object')
```

Renommez à nouveau la colonne **is_delay** en *target*. Utilisez le même code que celui que vous avez utilisé précédemment.

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# Enter your code here

Créez à nouveau les ensembles d'entraînement.

**Astuce :** utilisez la fonction `split_data` que vous avez définie (et utilisée) précédemment.

In [ ]:
# Entrez votre code ici

### Nouveau classificateur de base

Voyons maintenant si ces nouvelles fonctionnalités ajoutent un pouvoir prédictif au modèle.

In [ ]:
# Instancier l'objet estimateur LinearLearner
classifier_estimator2 = # Enter your code here

### Exemple de code

```
num_classes = len(pd.unique(train_labels))
classifier_estimator2 = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
instance_count=1,
instance_type='ml.m4.xlarge',
predictor_type='binary_classifier',
binary_classifier_model_selection_criteria = 'cross_entropy_loss')
```

In [ ]:
train_records = classifier_estimator2.record_set(train.values[:, 1:].astype(np.float32), train.values[:, 0].astype(np.float32), channel='train')
val_records = classifier_estimator2.record_set(validate.values[:, 1:].astype(np.float32), validate.values[:, 0].astype(np.float32), channel='validation')
test_records = classifier_estimator2.record_set(test.values[:, 1:].astype(np.float32), test.values[:, 0].astype(np.float32), channel='test')

Entraînez votre modèle en utilisant les trois ensembles de données que vous venez de créer.

In [ ]:
# Entrez votre code ici

Effectuez une prédiction par lots en utilisant le modèle nouvellement formé.

In [ ]:
# Entrez votre code ici

Tracer une matrice de confusion.

In [ ]:
# Entrez votre code ici

Le modèle linéaire ne présente qu'une légère amélioration des performances. Essayez un modèle d'ensemble basé sur un arbre, appelé *XGBoost*, avec Amazon SageMaker.

### Essayez le modèle XGBoost

Procédez comme suit :

1. Utilisez les variables de l'ensemble d'entraînement et enregistrez-les sous forme de fichiers CSV : train.csv, validation.csv et test.csv.
2. Stockez le nom du compartiment dans la variable. Le nom du compartiment Amazon S3 est fourni à gauche des instructions du laboratoire.
a. `bucket = <LabBucketName>`
b. `prefix = 'flight-xgb'`
3. Utilisez le kit AWS SDK pour Python (Boto3) pour charger le modèle dans le compartiment.

In [ ]:
bucket='c124417a3052650l8410971t1w16638019766-flightbucket-vr3vropvznlv'
prefix='flight-xgb'
train_file='flight_train.csv'
test_file='flight_test.csv'
validate_file='flight_validate.csv'
whole_file='flight.csv'
s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, folder, dataframe):
    csv_buffer = io.StringIO()
    dataframe.to_csv(csv_buffer, header=False, index=False )
    s3_resource.Bucket(bucket).Object(os.path.join(prefix, folder, filename)).put(Body=csv_buffer.getvalue())

upload_s3_csv(train_file, 'train', train)
upload_s3_csv(test_file, 'test', test)
upload_s3_csv(validate_file, 'validate', validate)

Utilisez la fonction `sagemaker.inputs.TrainingInput` pour créer un `record_set` pour les ensembles de données de formation et de validation.

In [ ]:
train_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train/".format(bucket,prefix,train_file),
    content_type='text/csv')

validate_channel = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validate/".format(bucket,prefix,validate_file),
    content_type='text/csv')

data_channels = {'train': train_channel, 'validation': validate_channel}

In [ ]:
from sagemaker.image_uris import retrieve
container = retrieve('xgboost',boto3.Session().region_name,'1.0-1')

In [ ]:
sess = sagemaker.Session()
s3_output_location="s3://{}/{}/output/".format(bucket,prefix)

xgb = sagemaker.estimator.Estimator(container,
                                    role = sagemaker.get_execution_role(),
                                    instance_count=1,
                                    instance_type=instance_type,
                                    output_path=s3_output_location,
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric = "auc",
                        num_round=100)

xgb.fit(inputs=data_channels)

Utilisez le transformateur par lots pour votre nouveau modèle et évaluez le modèle sur l’ensemble de données de test.

In [ ]:
batch_X = test.iloc[:,1:];
batch_X_file='batch-in.csv'
upload_s3_csv(batch_X_file, 'batch-in', batch_X)

In [ ]:
batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = xgb.transformer(instance_count=1,
                                       instance_type=instance_type,
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait()

Obtenez la cible prévue et les étiquettes de test.

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),sep=',',names=['target'])
test_labels = test.iloc[:,0]

Calculez les valeurs prédites en fonction du seuil défini.

**Remarque :** la cible prédite sera un score, qui doit être converti en classe binaire.

In [ ]:
print(target_predicted.head())

def binary_convert(x):
    threshold = 0.55
    if x > threshold:
        return 1
    else:
        return 0

target_predicted['target'] = target_predicted['target'].apply(binary_convert)

test_labels = test.iloc[:,0]

print(target_predicted.head())

Tracez une matrice de confusion pour vos « target_predicted » et « test_labels ».

In [ ]:
# Entrez votre code ici

### Essayez différents seuils

**Question** : En fonction de la façon dont le modèle a géré l’ensemble de tests, que pouvez-vous conclure ?

In [ ]:
# Entrez votre réponse ici

### Optimisation des hyperparamètres (HPO)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# ## Vous pouvez lancer plusieurs instances pour effectuer une optimisation des hyperparamètres en parallèle

xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker.get_execution_role(),
                                    instance_count= 1, # make sure you have a limit set for these instances
                                    instance_type=instance_type,
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(eval_metric='auc',
                        objective='binary:logistic',
                        num_round=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

hyperparameter_ranges = {'alpha': ContinuousParameter(0, 1000, scaling_type='Linear'),
                         'eta': ContinuousParameter(0.1, 0.5, scaling_type='Linear'),
                         'min_child_weight': ContinuousParameter(3, 10, scaling_type='Linear'),
                         'subsample': ContinuousParameter(0.5, 1),
                         'num_round': IntegerParameter(10,150)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10, # Set this to 10 or above depending upon budget and available time.
                            max_parallel_jobs=1)

In [ ]:
tuner.fit(inputs=data_channels)
tuner.wait()

<i class="fas fa-exclamation-triangle" style="color:red"></i> Attendez que la tâche de formation soit terminée. Cela peut prendre 25 à 30 minutes.

**Pour surveiller les tâches d'optimisation des hyperparamètres :**

1. Dans la console de gestion AWS, dans le menu **Services**, choisissez **Amazon SageMaker**.
2. Choisissez **Formation > Tâches de réglage des hyperparamètres**.
3. Vous pouvez vérifier l'état de chaque tâche de réglage des hyperparamètres, sa valeur de métrique objective et ses journaux.

Vérifiez que le travail a été effectué avec succès.

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

La tâche de réglage des hyperparamètres disposera d'un modèle qui a fonctionné le mieux. Vous pouvez obtenir les informations sur ce modèle à partir de la tâche de réglage.

In [ ]:
sage_client = boto3.Session().client('sagemaker')
tuning_job_name = tuner.latest_tuning_job.job_name
print(f'tuning job name:{tuning_job_name}')
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)
best_training_job = tuning_job_result['BestTrainingJob']
best_training_job_name = best_training_job['TrainingJobName']
print(f"best training job: {best_training_job_name}")

best_estimator = tuner.best_estimator()

tuner_df = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name).dataframe()
tuner_df.head()

Utilisez l'estimateur « best_estimator » et entraînez-le en utilisant les données.

**Astuce :** Consultez la fonction d'ajustement de l'estimateur XGBoost précédente.

In [ ]:
# Entrez votre code ici'

Utilisez le transformateur par lots pour votre nouveau modèle et évaluez le modèle sur l’ensemble de données de test.

In [ ]:
batch_output = "s3://{}/{}/batch-out/".format(bucket,prefix)
batch_input = "s3://{}/{}/batch-in/{}".format(bucket,prefix,batch_X_file)

xgb_transformer = best_estimator.transformer(instance_count=1,
                                       instance_type=instance_type,
                                       strategy='MultiRecord',
                                       assemble_with='Line',
                                       output_path=batch_output)

xgb_transformer.transform(data=batch_input,
                         data_type='S3Prefix',
                         content_type='text/csv',
                         split_type='Line')
xgb_transformer.wait()

In [ ]:
s3 = boto3.client('s3')
obj = s3.get_object(Bucket=bucket, Key="{}/batch-out/{}".format(prefix,'batch-in.csv.out'))
target_predicted = pd.read_csv(io.BytesIO(obj['Body'].read()),sep=',',names=['target'])
test_labels = test.iloc[:,0]

Obtenez la cible prévue et les étiquettes de test.

In [ ]:
print(target_predicted.head())

def binary_convert(x):
    threshold = 0.55
    if x > threshold:
        return 1
    else:
        return 0

target_predicted['target'] = target_predicted['target'].apply(binary_convert)

test_labels = test.iloc[:,0]

print(target_predicted.head())

Tracez une matrice de confusion pour vos « target_predicted » et « test_labels ».

In [ ]:
# Entrez votre code ici

**Question** : Essayez différents hyperparamètres et plages d'hyperparamètres. Ces modifications améliorent-elles le modèle ?

## Conclusion

Vous avez maintenant effectué au moins deux itérations dans la formation et l'évaluation de votre modèle. Il est temps de conclure ce projet et de réfléchir à :

- Ce que vous avez appris
- Quels types d'étapes vous pourriez suivre à l'avenir (en supposant que vous ayez plus de temps)

Utilisez la cellule suivante pour répondre à certaines de ces questions et à d'autres questions pertinentes :

1. Les performances de votre modèle répondent-elles à votre objectif commercial ? Si ce n'est pas le cas, quelles sont les choses que vous aimeriez faire différemment si vous aviez plus de temps pour les ajuster ?

2. Dans quelle mesure votre modèle s'est-il amélioré à mesure que vous apportiez des modifications à votre ensemble de données, à vos fonctionnalités et à vos hyperparamètres ? Quels types de techniques avez-vous utilisées tout au long de ce projet et lesquelles ont apporté les plus grandes améliorations à votre modèle ?

3. Quels ont été les plus grands défis que vous avez rencontrés tout au long de ce projet ?

4. Avez-vous des questions sans réponse sur des aspects du pipeline qui n'avaient pas de sens pour vous ?

5. Quelles ont été les trois choses les plus importantes que vous avez apprises sur l'apprentissage automatique en travaillant sur ce projet ?

#### <span style="color: blue;">Présentation du projet : veillez à résumer également vos réponses à ces questions dans votre présentation du projet. Rassemblez toutes vos notes pour votre présentation du projet et préparez-vous à présenter vos conclusions à la classe.</span>

In [ ]:
# Écrivez vos réponses ici